Размышления в вольном стиле.
При первом взгляде на задачу в памяти всплывает словосочетание "временные ряды".
В самом деле - имеем данные ("наблюдения") полученные через равные промежутки времени. Но наблюдаемых объектов много - т.е. в каждый момент времени имеем несколько ("много") значений.
Т.е. уже не временной ряд а "panel data". Которые панельные данне анализировать посложнее чем временной ряд. С другой стороны проще всё таки рассмотреть данные по каждому из пользователей как независимый
от других времменной ряд. Но независимый ли? Но предположим, что независимый - в самом деле как зависит оплата за услуги одного пользователя от оплаты другим, и будем что то делать с данными по каждому пользователю (которые данные - попросту некоторая последовательность чисел) независимо от данных по всем другим. И тут самая главная проблема - моментов времени в которые производились "наблюдения" немного, <= 12. Т.е. о каких то "продвинутых" техниках работы с временными рядами, выявления периодической, сезонной частей говорить не приходится. С трудом можно (наверное) использовать скользящее среднее. Напрашивается вывод что "временнорядные" техники работы с этими данными нам практически без надобности.
Тогда как будем строить prediction?  Простейший вариант - считать оплату линейной функцией времени. Ну и использовать линейную регрессию для нахождения оной. По каждому пользователю. А если оплата им, пользователем, услуг от месяца к месяцу сильно колеблеться? Использовать нелинейную функцию времени в качестве аппроксимирующей? Интуиция подсказывает, что в данном случае это не есть хорошее решение. Ну хотя бы, вот такое рассуждение "на глазок": для "хорошего" пользователя - у которого оплата не "скачет" сильно - и линейная регрессия сойдёт, а для "нехорошего" пользователя придется подбирать другую причём "индивидуальную" аппроксимирующую функцию (напр. полином степени отличной от 1-й). Ключевое слово здесь "индивидуальная" - т.е. для каждого пользователя надо подобрать аппроксимирующую функцию. Для каждого! Нет, можно, конечно поступить проще - воспользоваться какой нибудь интерполяционной формулой (напр. Лагранжа) и найти многочлен который точно "попадёт" во все точки на плоскости "время - оплата" с независимой переменной "время". Но что то подсказывает что это решение совсем никуда не годится (Да и вообще это не задача интерполяции - время у нас дискретно, промежуточных значений между точками времени "март" и "апрель" нет, ). Опять же, сумму оплаты по всем пользователям на следующий месяц скорее всего можно довольно точно предсказать воспользовавшись линейной регрессией (А были бы данные за "большой" промежуток времени или точнее "много" (> 100)  моментов времени в которые производились наблюдения, то можно было бы и time series forecasting применить). Но это несколько не то что нам нужно. Нужно по каждому пользователю дать prediction.
Ну, вообщем, и так уже много написанно, так что лучше сразу перейти к идее решения. Будем рассматривать какую-то часть (подпоследовательность то есть несколько последовательных значений) набора данных по каждому "пользователю" в качестве независимых переменных а значение (неизвестное) которое необходимо "узнать" - в качестве зависимой переменной. В предельном варианте вся известные N значений по каждому пользователю являються конкретными значениями N независимых переменных а неизвестное значение величины оплаты в следующий месяц  - зависимой переменной.  И соответственно, есть некая функция которая на основе данных значений независимых переменных вычисляет зависимую - "следующее" значение оплаты. Тут сразу стоит заметить что в данном случае мы, так сказать, опускаем за ненадобностью "временнУю" природу данных.
Осталось дело за малым - найти эту функцию от n <= N переменных.Ну а это уже отчётливо поставленная задача машинного обучения. Надо построить нейронную сеть (т.к. при помощи таковой можно аппроксимировать практически любую функцию) принимающую на вход "предыдущие" значения оплаты и выдающую на выходе "последующее" значение. Для тренировки такой сети можно брать подпоследовательность значений оплаты, например за январь, февраль, ..., июль и вычислять значение за август (которое у нас уже имеется. Положим что у нас есть значения с января по август). Далее "скормить" натренированной сети значения за февраль - август и получить значение за сентябрь которое нам и нужно предсказать. С длинной тренировочной подпоследовательности можно поэкспериментировать. И наверное, стоило бы поэкспериментировать с реккурентными нейронными сетями, но здесь я их использовать не буду.
И последнее. Само собой нам не важно с какими данными в данном случае надо иметь дело - с помесячной оплатой пользователя за объём отправленной почты, или с годовым удоем молока по фермам.

In [155]:
import sys
import warnings
import pickle

import numpy as np
import pandas as pd
from pandas import Series, DataFrame

from numpy.testing import assert_almost_equal, assert_array_equal

from sklearn import cross_validation
from sknn.mlp import Regressor, Layer
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.grid_search import  GridSearchCV
from sklearn.utils.testing import (assert_raises, assert_greater,
                                   assert_equal, assert_false)

In [156]:
np.seterr(all='warn')
scaler = StandardScaler()

In [157]:
payment_data = pd.read_csv('user_payments.csv', index_col=0)
payment_data.head(4)

,January,February,March,April,May,June
User,,,,,,
1,73.65,91.26,118.50,111.45,110.42,131.20
2,118.15,127.74,146.49,163.43,163.60,169.54
3,102.44,129.94,134.28,132.59,140.58,182.04
4,405.04,489.13,526.05,545.83,570.26,543.84


Как мы видим, данных у нас всего за шесть месяцев что гораздо меньше 12

In [158]:
payment_data_matrix = payment_data.as_matrix()
payment_data_matrix = payment_data_matrix.astype(np.float64)

Для удобства всё связанное с обучением нашего multi layer perceptron-а выносим в отдельную функцию.

In [159]:
def train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.002, n_iter=20, units=4):
    print learning_rule
    mlp = Regressor(layers=[Layer('Rectifier', units=units),
                            Layer('Linear')],
                   learning_rule=learning_rule,
                   learning_rate=learning_rate,
                   n_iter=n_iter)
    mlp.fit(train_x, train_y)
    print mlp.score(train_x, train_y)
    return mlp

In [160]:
#Выделяем наши независимые переменны.
#Это данные по каждому из пользователей с января по апрель
#(включительно), всего четыре переменные.
independed_vars_values = payment_data_matrix[0:,:4]

In [161]:
#Выделяем независимую переменную - данные за май
depended_var_values = payment_data_matrix[0:, 4]

In [162]:
train_x, test_x, train_y, test_y = cross_validation.train_test_split(independed_vars_values, depended_var_values, test_size=0.3)

In [163]:
train_x = scaler.fit_transform(train_x)
test_x = scaler.fit_transform(test_x)

In [164]:
payment_mlp = train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.0002, n_iter=20, units=4)

sgd
0.937940194241


In [165]:
pickle.dump(payment_mlp, open('payment_mlp.pkl', 'wb'))
payment_mlp.score(test_x, test_y)

0.95152056161421461

In [166]:
# Выделяем "реальные" (февраль-май) значения независмых переменных
# на которых обученная нейронная сеть будет вычислять оплату
# на следующий месяц (июнь) 
real_x = payment_data_matrix[0:, 1:5]
real_x = scaler.fit_transform(real_x)

In [167]:
# А вот пришли "реальные" данные за июнь
real_y = payment_data_matrix[0:, 5]

In [168]:
# Смотрим, насколько же хорошо/плохо мы предсказали оплату
payment_mlp.score(real_x, real_y)

0.91176690095007806

In [169]:
# Немного покроссвалидируем

for i, (train_indices, test_indices) in enumerate(cross_validation.KFold(depended_var_values.size, n_folds=3, shuffle=True)):
    train_x = independed_vars_values[train_indices]
    test_x = independed_vars_values[test_indices]
    train_y = depended_var_values[train_indices]
    test_y = depended_var_values[test_indices]
    train_x = scaler.fit_transform(train_x)
    test_x = scaler.fit_transform(test_x)
    payment_mlp = train_regression_predictor(train_x, train_y, learning_rule='sgd', learning_rate=0.0002, n_iter=20, units=4)
    pickle.dump(payment_mlp, open('payment_mlp_cv_0%s_sgd.pkl' % str(i), 'wb'))
    print payment_mlp.score(test_x, test_y)
    print payment_mlp.score(real_x, real_y)

sgd
0.942918753604
0.965288910981
0.923921970727
sgd
0.901430093541
0.93980591232
0.950398089974
sgd
0.927359442428
0.895740701383
0.861471255602


In [170]:
predicted_y = payment_mlp.predict(real_x)

In [171]:
predicted_and_true_values = DataFrame({'predicted' : list(predicted_y), 'real' : list(real_y)})
predicted_and_true_values.head(20)

,predicted,real
0,[105.645317754],131.20
1,[151.024713759],169.54
2,[129.824128603],182.04
3,[464.297496872],543.84
4,[137.40570964],125.72
5,[139.285024939],169.95
6,[209.359552341],236.43
7,[167.77173989],216.86
8,[495.234180636],608.13
9,[250.917350702],283.80


In [172]:
#tuned_parameters = [{'kernel': ['rbf'], 'gamma': [1e-3, 1e-4],
#                     'C': [1, 10, 100, 1000]},
#                    {'kernel': ['linear'], 'C': [1, 10, 100, 1000]}]

In [173]:
#scores = ['precision', 'recall']
#for score in scores:
#    print('Tuning parameters for %s' % score)
#    print()
#    clf = GridSearchCV(SVC(C=1), tuned_parameters, cv=5, scoring='%s_weighted' % score)
#    clf.fit(X_train, y_train)
#    print('Best parameters set found on development set:')
#    print()
#    print(clf.best_params_)
#    print()
#    print('Grid scores on development set:')
#    print()
#    for params, mean_score, scores in clf.grid_scores_:
#        print('%0.3f (+/-%0.03f) for %r') % (mean_score, scores.std() * 2, params))
#    print()
#    print('Detailed classifcation report:')
#    print()
#    print('The model is trined on the full development set.')
#    print('The scores are computed on the full evaluation set.')
#    print()
#    y_true, y_pred = y_test, cls.predict(X_test)
#    print(classification_report(y_true, y_pred))
#    print()